In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string

%matplotlib inline

In [108]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Cleaning CFB Stats (scraped), calculating some features, creating timeseries features
* Data comes from two csvs - scraped passing and scraped rushing/receiving yards
* All CFB players must play at least two years
    * Get last two years of stats, plus the difference betwen the two
* Clean up NaN values as required to zero in stats fields
* Calculate player total games

In [109]:
df_pass = pd.read_csv('cfb_qb_passing.csv')

In [110]:
del df_pass['Unnamed: 0']
del df_pass['adj_pass_yds_per_att']
del df_pass['pass_cmp_pct']
del df_pass['pass_rating']
del df_pass['pass_yds_per_att']
del df_pass['class']


In [111]:
# clean up years
df_pass['year_id'] = df_pass.apply(lambda x: int(x['year_id'].replace('*','')), axis=1)

#clean up NaN
df_pass.fillna(0,inplace=True)

In [112]:
#get school graduated from, grad year, # years
df_grad = df_pass.groupby('player', as_index=False).agg({'year_id':'max', 'pos':'count'})

df_grad.rename(columns={'pos': 'years'}, inplace=True)

In [113]:
# get CFB player last year stats
df_pass_last = df_pass.merge(df_grad, on = ['player','year_id'])

del df_pass_last['pos']


In [114]:
#get cfb player total games
df_cfb_games = df_pass.groupby('player', as_index=False)['g'].sum()
df_cfb_games.rename(columns={'g':'cfb_games'},inplace=True)

In [115]:
# import rushing data
df_rush = pd.read_csv('cfb_qb_rushing.csv')

In [116]:
cols_to_save = [4,5,6,7,8,9,10,11,12,19]
df_rushc = df_rush.iloc[:,cols_to_save].copy()
del df_rushc['pos']
del df_rushc['rec_yds_per_rec']
df_rushc.fillna(0,inplace=True)
df_rushc['year_id'] = df_rushc.apply(lambda x: int(x['year_id'].replace('*','')), axis=1)

In [117]:
#combine all the dataframes
df_comb = df_pass_last.merge(df_rushc, how='left', on=['player','year_id'])
df_comb = df_comb.merge(df_cfb_games, on='player')
del df_comb['g']

In [118]:
df_comb.fillna(0, inplace=True)

In [119]:
# pickle
import pickle

with open('cfb_grad_stats.pkl', 'wb') as picklefile:
    pickle.dump(df_comb, picklefile)

In [120]:
# only players that have played >2 years
df_2yr_p = df_grad[df_grad.years > 1]

In [121]:
# getting only players that have played >2 years
# left join to show NaNs(will remove if not NaN (i.e. remove  grad year))
# get 2nd to last year
# maybe we could have subtracted 1 from grad year, but that feels dangerous to me
df_pass_2yr = df_pass.merge(df_2yr_p, on='player')
df_NaNs = df_pass_2yr.merge(df_2yr_p, left_on=['player','year_id_x'], right_on=['player','year_id'], how='left')
df_NaNs = df_NaNs[pd.isnull(df_NaNs.year_id)]
df_2nd = df_NaNs.groupby('player', as_index=False).agg({'year_id_x':'max'})

In [122]:
# get second year passing stats
df_pass_2nd = df_pass.merge(df_2nd, left_on = ['player','year_id'], right_on = ['player','year_id_x'])

del df_pass_2nd['conf_abbr']
del df_pass_2nd['g']
del df_pass_2nd['pos']
del df_pass_2nd['school_name']
del df_pass_2nd['year_id']
del df_pass_2nd['year_id_x']

# change column names for 2nd year
df_pass_2nd.columns = df_pass_2nd.columns + "_2"


In [123]:
# get second year rushing stats
df_rush_2nd = df_rushc.merge(df_2nd, left_on = ['player','year_id'], right_on = ['player','year_id_x'])

del df_rush_2nd['year_id']
del df_rush_2nd['year_id_x']

# change column names for 2nd year
df_rush_2nd.columns = df_rush_2nd.columns + "_2"

In [125]:
#combine 2nd year rushing & passsing

df_2nd = df_rush_2nd.merge(df_pass_2nd, on='player_2')

In [126]:
# combine 2nd year rushing and passing w/ other data
df_comb = df_comb.merge(df_2nd, left_on = 'player', right_on = 'player_2')
del df_comb['player_2']

In [128]:
#calculate difference between 1st and 2nd year
cols_to_calc = ['pass_att', 'pass_cmp', 'pass_int', 'pass_td', 'pass_yds',
               'rec', 'rec_td', 'rec_yds', 'rush_att', 'rush_td', 'rush_yds']

for col in cols_to_calc:
    df_comb[col + '_diff'] = df_comb.apply(lambda x: x[col] - x[col + '_2'], axis=1)

In [132]:
with open('cfb_grad_stats_2yr.pkl', 'wb') as picklefile:
    pickle.dump(df_comb, picklefile)